In [3]:
from yahoo_oauth import OAuth2
import os
import json
import league, game, team
import yhandler
import json
import objectpath
import pandas as pd
from pandas.io.json import json_normalize

In [66]:
# get our session context ("sc")
oauth = OAuth2(None, None, from_file='oauth2.json')

[2019-08-14 22:52:08,158 DEBUG] [yahoo_oauth.yahoo_oauth.__init__] Checking 
[2019-08-14 22:52:08,167 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 1282.521208524704
[2019-08-14 22:52:08,168 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN IS STILL VALID


In [67]:
# use package to get game
gm = game.Game(oauth, 'nfl')
print(gm)

In [11]:
#use package to get league_id
league_ids = gm.league_ids()
league_id = league_ids[0]
league_id

'390.l.707700'

In [12]:
#get league
lg = gm.to_league(league_id)
print(f"scoring categories: {lg.stat_categories()}")
print(f"standings:{lg.standings()}")
print(f"teams:{lg.teams()}")
print(f"current user team:{lg.team_key()}")
print(f"current week:{lg.current_week()}")

scoring categories: [{'display_name': 'Pass Yds', 'position_type': 'O'}, {'display_name': 'Pass TD', 'position_type': 'O'}, {'display_name': 'Rush Yds', 'position_type': 'O'}, {'display_name': 'Rush TD', 'position_type': 'O'}, {'display_name': 'Rec Yds', 'position_type': 'O'}, {'display_name': 'Rec TD', 'position_type': 'O'}, {'display_name': '2-PT', 'position_type': 'O'}, {'display_name': 'Fum Ret TD', 'position_type': 'O'}, {'display_name': 'FG 0-19', 'position_type': 'K'}, {'display_name': 'FG 20-29', 'position_type': 'K'}, {'display_name': 'FG 30-39', 'position_type': 'K'}, {'display_name': 'FG 40-49', 'position_type': 'K'}, {'display_name': 'FG 50+', 'position_type': 'K'}, {'display_name': 'PAT Made', 'position_type': 'K'}]
standings:['Resident Stevil', 'Dude in Omaha', 'Hair Force Blue', 'Hawkeyes', 'Healthy Scratch', 'KnightHawks', 'Menty', 'Monarchs', 'Redskins', 'War Machine', 'Werkmonsters', 'Wedgie Woty']
teams:[{'name': 'Resident Stevil', 'team_key': '390.l.707700.t.1'}, {'

In [13]:
# get league teams:
teams = lg.teams()
teams

[{'name': 'Resident Stevil', 'team_key': '390.l.707700.t.1'},
 {'name': 'Dude in Omaha', 'team_key': '390.l.707700.t.2'},
 {'name': 'Hair Force Blue', 'team_key': '390.l.707700.t.3'},
 {'name': 'Hawkeyes', 'team_key': '390.l.707700.t.4'},
 {'name': 'Healthy Scratch', 'team_key': '390.l.707700.t.5'},
 {'name': 'KnightHawks', 'team_key': '390.l.707700.t.6'},
 {'name': 'Menty', 'team_key': '390.l.707700.t.7'},
 {'name': 'Monarchs', 'team_key': '390.l.707700.t.8'},
 {'name': 'Redskins', 'team_key': '390.l.707700.t.9'},
 {'name': 'War Machine', 'team_key': '390.l.707700.t.10'},
 {'name': 'Werkmonsters', 'team_key': '390.l.707700.t.11'},
 {'name': 'Wedgie Woty', 'team_key': '390.l.707700.t.12'}]

In [14]:
for d in teams:
    print(d["team_key"])

390.l.707700.t.1
390.l.707700.t.2
390.l.707700.t.3
390.l.707700.t.4
390.l.707700.t.5
390.l.707700.t.6
390.l.707700.t.7
390.l.707700.t.8
390.l.707700.t.9
390.l.707700.t.10
390.l.707700.t.11
390.l.707700.t.12


In [15]:
# testing manual pulling of data
YAHOO_ENDPOINT = 'https://fantasysports.yahooapis.com/fantasy/v2'
uri = 'game/nfl//'

In [16]:
response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, uri),params={'format': 'json'})

In [17]:
jresp = response.json()
jresp

{'fantasy_content': {'xml:lang': 'en-US',
  'yahoo:uri': '/fantasy/v2/game/nfl/',
  'game': [{'game_key': '390',
    'game_id': '390',
    'name': 'Football',
    'code': 'nfl',
    'type': 'full',
    'url': 'https://football.fantasysports.yahoo.com/f1',
    'season': '2019',
    'is_registration_over': 0,
    'is_game_over': 0,
    'is_offseason': 0,
    'is_live_draft_lobby_active': 1}],
  'time': '10.309934616089ms',
  'copyright': 'Data provided by Yahoo! and STATS, LLC',
  'refresh_rate': '60'}}

In [18]:
url = "users;use_login=1/games/teams"
response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, url),params={'format': 'json'})

In [19]:
# prepare functions
def get_json_dump(url):
    YAHOO_ENDPOINT = 'https://fantasysports.yahooapis.com/fantasy/v2'
    response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, url),params={'format': 'json'})
    jresp = response.json()
    return json.dumps(jresp)
def get_json(url):
    YAHOO_ENDPOINT = 'https://fantasysports.yahooapis.com/fantasy/v2'
    response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, url),params={'format': 'json'})
    jresp = response.json()
    return jresp
def get_json_t(url):
    YAHOO_ENDPOINT = 'https://fantasysports.yahooapis.com/fantasy/v2'
    response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, url),params={'format': 'json'})
    jresp = response.json()
    t = objectpath.Tree(jresp)
    return t

In [20]:
myteamsurl = "users;use_login=1/games/teams"
myteamsjson = get_json(myteamsurl)
# myteams = myteamsjson.jsonify()
myteamsjson

{'fantasy_content': {'xml:lang': 'en-US',
  'yahoo:uri': '/fantasy/v2/users;use_login=1/games/teams',
  'users': {'0': {'user': [{'guid': 'CCOCHI6ILR3M3O6HT5PTS5KN5U'},
     {'games': {'0': {'game': [{'game_key': '390',
          'game_id': '390',
          'name': 'Football',
          'code': 'nfl',
          'type': 'full',
          'url': 'https://football.fantasysports.yahoo.com/f1',
          'season': '2019',
          'is_registration_over': 0,
          'is_game_over': 0,
          'is_offseason': 0,
          'is_live_draft_lobby_active': 1},
         {'teams': {'0': {'team': [[{'team_key': '390.l.707700.t.5'},
              {'team_id': '5'},
              {'name': 'Healthy Scratch'},
              {'is_owned_by_current_login': 1},
              {'url': 'https://football.fantasysports.yahoo.com/f1/707700/5'},
              {'team_logos': [{'team_logo': {'size': 'large',
                  'url': 'https://s.yimg.com/cv/apiv2/default/nfl/nfl_4.png'}}]},
              {'division

In [21]:
myteamsjson["fantasy_content"]["users"]["0"]["user"][1]["games"]["0"]["game"][1]["teams"]["0"]["team"][0][2]["name"]

'Healthy Scratch'

In [22]:
myteamt = get_json_t(myteamsurl)
jfilter = myteamt.execute('$..(team_key,season,code, name)')
print(jfilter)

<generator object filter_dict at 0x00000175D13362A0>


In [23]:
for row in jfilter:
    print(row)

{'season': '2019', 'code': 'nfl', 'name': 'Football'}
{'team_key': '390.l.707700.t.5'}
{'name': 'Healthy Scratch'}


In [24]:
league_id = '390.l.707700'
# get my teams: "users;use_login=1/games/teams"
# get standings: f"league/{league_id}/standings"
# get settings: f"league/{league_id}/settings"
# get matchups: f"league/{league_id}/settings"
# get roster: f"team/{team_key}/roster;week={week}"
# get scoreboard: f"league/{league_id}/scoreboard;week={week}"
# get all teams: "users/games/teams"

In [25]:
oneteamurl = f"/league/{league_id}/teams"
onet = get_json(oneteamurl)
onet = onet["fantasy_content"]["league"][1]["teams"]["0"]["team"][0]
onet

[{'team_key': '390.l.707700.t.1'},
 {'team_id': '1'},
 {'name': 'Resident Stevil'},
 [],
 {'url': 'https://football.fantasysports.yahoo.com/f1/707700/1'},
 {'team_logos': [{'team_logo': {'size': 'large',
     'url': 'https://ct.yimg.com/cy/1520/25313688185_4b0ea69cda_192sq.jpg?ct=fantasy'}}]},
 {'division_id': '1'},
 {'waiver_priority': ''},
 {'faab_balance': '100'},
 {'number_of_moves': 0},
 {'number_of_trades': 0},
 {'roster_adds': {'coverage_type': 'week',
   'coverage_value': '1',
   'value': '0'}},
 [],
 {'league_scoring_type': 'head'},
 [],
 [],
 {'has_draft_grade': 0},
 {'auction_budget_total': '200'},
 [],
 {'managers': [{'manager': {'manager_id': '1',
     'nickname': 'Steve W',
     'guid': 'PORZKU7UN5TJHAWJIOGS5FX6FM',
     'is_commissioner': '1',
     'email': 'skwichmer@gmail.com',
     'image_url': 'https://s.yimg.com/ag/images/4542/27255030819_6f4805_64sq.jpg'}}]}]

In [ ]:
onet = [item for item in onet if item != []]
onet

In [ ]:
onetdict = {k: v for d in onet for k, v in d.items()}
onetdict

In [28]:
# can't iterate directly over dictionary
for k,v in onetdict.items():
    if isinstance(onetdict[k], list):
        print("found a list")
        print(onetdict[k][0])
        onetdict[k] = onetdict[k][0]
onetdict

found a list
{'team_logo': {'size': 'large', 'url': 'https://ct.yimg.com/cy/1520/25313688185_4b0ea69cda_192sq.jpg?ct=fantasy'}}
found a list
{'manager': {'manager_id': '1', 'nickname': 'Steve W', 'guid': 'PORZKU7UN5TJHAWJIOGS5FX6FM', 'is_commissioner': '1', 'email': 'skwichmer@gmail.com', 'image_url': 'https://s.yimg.com/ag/images/4542/27255030819_6f4805_64sq.jpg'}}


{'team_key': '390.l.707700.t.1',
 'team_id': '1',
 'name': 'Resident Stevil',
 'url': 'https://football.fantasysports.yahoo.com/f1/707700/1',
 'team_logos': {'team_logo': {'size': 'large',
   'url': 'https://ct.yimg.com/cy/1520/25313688185_4b0ea69cda_192sq.jpg?ct=fantasy'}},
 'division_id': '1',
 'waiver_priority': '',
 'faab_balance': '100',
 'number_of_moves': 0,
 'number_of_trades': 0,
 'roster_adds': {'coverage_type': 'week', 'coverage_value': '1', 'value': '0'},
 'league_scoring_type': 'head',
 'has_draft_grade': 0,
 'auction_budget_total': '200',
 'managers': {'manager': {'manager_id': '1',
   'nickname': 'Steve W',
   'guid': 'PORZKU7UN5TJHAWJIOGS5FX6FM',
   'is_commissioner': '1',
   'email': 'skwichmer@gmail.com',
   'image_url': 'https://s.yimg.com/ag/images/4542/27255030819_6f4805_64sq.jpg'}}}

In [29]:
raw_team_data = json_normalize(onetdict, sep="_")
raw_team_data
# df = pd.DataFrame(alldict)
# df

,auction_budget_total,division_id,faab_balance,has_draft_grade,league_scoring_type,managers_manager_email,managers_manager_guid,managers_manager_image_url,managers_manager_is_commissioner,managers_manager_manager_id,...,number_of_trades,roster_adds_coverage_type,roster_adds_coverage_value,roster_adds_value,team_id,team_key,team_logos_team_logo_size,team_logos_team_logo_url,url,waiver_priority
0,200,1,100,0,head,skwichmer@gmail.com,PORZKU7UN5TJHAWJIOGS5FX6FM,https://s.yimg.com/ag/images/4542/27255030819_...,1,1,...,0,week,1,0,1,390.l.707700.t.1,large,https://ct.yimg.com/cy/1520/25313688185_4b0ea6...,https://football.fantasysports.yahoo.com/f1/70...,


In [ ]:
allteamurl = f"/league/{league_id}/teams"
allt = get_json(allteamurl)
allt = allt["fantasy_content"]["league"][1]["teams"]#["0"]["team"][0]
allt

In [ ]:
# allt = allt["fantasy_content"]["league"][1]["teams"]#["0"]["team"][0]
teamlist = []
for k, v in allt.items():
    if k != "count": #ignore count item which json adds as autosummary at the end
        team = allt[k]["team"][0] #go down to individual team level in json
        team = [item for item in team if item != []]     #remove all empty lists
        teamd = { k : v[0] if type(v)==list else v  for d in team for k, v in d.items()} #convert all dictionaries into 1 dictionary using comprehension, if they contain a list only use first item from list
        print(teamd)
        teamlist.append(teamd)
    
#     onet = [item for item in onet if item != []] #remove blank lists
# onetdict = {k: v for d in allt for k, v in d.items()}
# # can't iterate directly over dictionary
# for k,v in onetdict.items():
#     if isinstance(onetdict[k], list):
#         print("found a list")
#         print(onetdict[k][0])
#         onetdict[k] = onetdict[k][0]
# onetdict
    

In [ ]:
teamsdf = json_normalize(teamlist, sep="_") #this only gets the sub rows?
teamsdf

In [ ]:
dfteams = pd.DataFrame(teamlist)
dfteams


In [34]:
teamsdf.columns

Index(['auction_budget_total', 'division_id', 'faab_balance',
       'has_draft_grade', 'is_owned_by_current_login', 'league_scoring_type',
       'managers_manager_email', 'managers_manager_guid',
       'managers_manager_image_url', 'managers_manager_is_commissioner',
       'managers_manager_manager_id', 'managers_manager_nickname', 'name',
       'number_of_moves', 'number_of_trades', 'roster_adds_coverage_type',
       'roster_adds_coverage_value', 'roster_adds_value', 'team_id',
       'team_key', 'team_logos_team_logo_size', 'team_logos_team_logo_url',
       'url', 'waiver_priority'],
      dtype='object')

In [35]:
# allteamst = get_json_t(allteamsurl)
# allteams = allt["fantasy_content"]["league"][1]["teams"]
# mydictlist = []
# for key in allteams:
#     print(key)
#     print(allteams[key]["team"])
#     mydictlist.append(allteams[key]["team"])
# mydictlist

In [36]:
# df = pd.DataFrame(mydictlist) 
# df.head()

In [37]:
# jfilter = allteamst.execute('$..(name)')

# filter = f"$.fantasy_content.league.({})[0]"
# filter2 = allteamst.execute('$..league')
# for row in filter2:
#     print(row)
#     print("next team_________________________________________________")

In [65]:
#get team based on team id
teamid = "390.l.707700.t.1"
teamurl = f"/team/{teamid}"
teamdata = get_json_t(teamurl)
jfilter = teamdata.execute('$..(team_id,name, division_id, faab_balance, number_of_moves, number_of_trades, manager_id, email, nickname)')
print(jfilter)
# for d in jfilter:
#     print(r)
# #     {k: v for d in onet for k, v in d.items()}
# # teamdict = { k:v for d in jfilter}
# # teamdict = dict(jfilter)
# print(f"teamdict:{teamdict}")
filterlist = list(jfilter)
# teamd = { k : v[0] if type(v)==list else v  for d in filterlist for k, v in d.items()} 
teamd = {k: v for d in filterlist for k, v in d.items()}
teamd

In [65]:
def get_team_dict(teamid):
    teamurl = f"/team/{teamid}"
    teamdata = get_json_t(teamurl)
    #get the fields that we want
    jfilter = teamdata.execute('$..(team_id,name, division_id, faab_balance, number_of_moves, number_of_trades, manager_id, email, nickname)')
    filterlist = list(jfilter)
    teamd = {k: v for d in filterlist for k, v in d.items()}
    return teamd

<generator object filter_dict at 0x00000175D1C81048>


{'team_id': '1',
 'name': 'Resident Stevil',
 'division_id': '1',
 'faab_balance': '100',
 'number_of_moves': 0,
 'number_of_trades': 0,
 'manager_id': '1',
 'email': 'skwichmer@gmail.com',
 'nickname': 'Steve W'}